In [2]:
import json
from ast import literal_eval
from pprint import pprint

from bs4 import BeautifulSoup
import pandas as pd
import requests

---- we can search for a place and then return first hit for a search id ----

In [3]:
#use dict to search for listings
url = "https://www.hemnet.se/locations/show"
params= {
    'q':'Lilla Essingen, Stockholm'
}
payload={}

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.76 Safari/537.36'
}

response = requests.request("GET", url, headers=headers, data=payload, params=params)
#response.text
response.url

'https://www.hemnet.se/locations/show?q=Lilla+Essingen%2C+Stockholm'

url = "https://www.hemnet.se/locations/show"
params= {
    'q':'Gärdet, Stockholm'
}
next(iter(params.values()))
str=f"{url}?{next(iter(params))}={next(iter(params.values()))}"
print(str)

import urllib.parse
url = "https://www.hemnet.se/locations/show"
params= {
    'q':'Gärdet, Stockholm'
}
#params = {'q': 'Python 2.x URL encoding', 'as_sitesearch': 'www.urlencoder.io'}
urllib.parse.urlencode(params)

In [4]:
pprint(literal_eval(response.text))

[{'id': 473386,
  'location_type': 'district',
  'name': 'Lilla Essingen',
  'parent_location': {'id': 18031,
                      'location_type': 'municipality',
                      'name': 'Stockholms kommun',
                      'slug': 'stockholms-kommun'},
  'slug': 'lilla-essingen'}]


In [5]:
json_data=json.loads(response.text)
search_id=json_data[0]['id']

In [6]:
search_id

473386

In [7]:
#or make dict with places to search for

#location_ids={'Ringvägen':475989,'Huvudsta':473498}
#search_id=location_ids['Ringvägen']

In [8]:
#use dict to search for listings
url = "https://www.hemnet.se/bostader"
params= {
    'housing_form_groups':'apartments',
    'location_ids':search_id,
    'item_types':'bostadsratt',
    'rooms_min':2,
    'living_area_min':35,
    'new_construction':'exclude'
}
payload={}

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.76 Safari/537.36'
}

response = requests.request("GET", url, headers=headers, data=payload, params=params)
response.url

'https://www.hemnet.se/bostader?housing_form_groups=apartments&location_ids=473386&item_types=bostadsratt&rooms_min=2&living_area_min=35&new_construction=exclude'

--------- w beautiful soup ------------

In [9]:
soup = BeautifulSoup(response.content, "html.parser")

In [10]:
#find div with good data to use
map_results=soup.find(id='results-map')
#map_results

In [11]:
initial_data=map_results.attrs['data-initial-data']
#initial_data

In [12]:
#find our search key to use
json_data=json.loads(initial_data)
#pprint(json_data)

In [13]:
json_data['search_key']

'9ccab53fe070c5b95c2fa417d46cb2f5652d6e3a'

In [14]:
url = "https://www.hemnet.se/bostader/search/"+json_data['search_key']

params= {}

payload={}

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.76 Safari/537.36'
}

response = requests.request("GET", url, headers=headers, data=payload, params=params)

#print(response.text)

In [15]:
#print(response.text)

In [16]:
r_json = response.json()

#pprint(r_json)

In [17]:

r_prop = r_json['properties']
#r_prop

In [18]:
#this output should prob be filtered some to drop crappy columns
df=pd.json_normalize(r_prop, max_level=1)
df.head(3)

,id,coordinate,address,location_name,typeSummary,iconName,project,new_construction,ongoing_bidding,foreclosure,...,locations_string,should_display_showings,should_highlight_showings,living_space.table,trackingData.listingId,trackingData.listingType,trackingData.listingPackageType,trackingData.listingState,trackingData.listingStatus,price_change
0,19326036,"[59.3239, 18.006191]",Strålgatan 19,Stockholms kommun,Bostadsrättslägenhet,bostadsratt,False,False,False,False,...,"Lilla Essingen - Kungsholmen, Stockholms kommun",False,False,"{'living_area': '58 m²', 'supplemental_area': ...",19326036,common_listing,not_selected,active,for_sale,NaN
1,19325274,"[59.32441, 18.005714]","Strålgatan 8, 2 tr",Stockholms kommun,Bostadsrättslägenhet,bostadsratt,False,False,False,False,...,"Lilla Essingen - Kungsholmen, Stockholms kommun",False,False,"{'living_area': '54 m²', 'supplemental_area': ...",19325274,common_listing,not_selected,active,for_sale,NaN
2,19277882,"[59.3244369, 18.0033101]","Strålgatan 45, 3 tr",Stockholms kommun,Bostadsrättslägenhet,bostadsratt,False,False,False,False,...,"Kungsholmen - Lilla Essingen, Stockholms kommun",False,False,"{'living_area': '42 m²', 'supplemental_area': ...",19277882,common_listing,basic,active,for_sale,NaN


In [19]:
df.shape

(14, 39)

In [20]:
#call graphql directly from listing with ids - example with one id only but can set into a loop
ql_id=df.iloc[0]['id']
ql_id

19326036

In [21]:
url_ql = "https://www.hemnet.se/graphql"

#would love to have payload_ql string formatted better but not really sure how without breaking it
payload_ql="{\"query\":\"query imagesForListing($id: ID!) {\\r\\n    listing(id: $id) \\r\\n    {  id  __typename  streetAddress  isSaved  brokerAgency \\r\\n    {    name    id    brokerCustomization \\r\\n    {      compactLogoUrl      largeLogoUrl: logoUrl(format: BROKER_CUSTOMIZATION_LARGE)      __typename    \\r\\n    }    __typename  }  ... on ActivePropertyListing \\r\\n    {    liveStreams(scope: ENDED) \\r\\n    {      embedUrl      __typename    \\r\\n    }    videoAttachment: attachment(type: VIDEO) \\r\\n    {      id      attachmentType      ... on VideoAttachment {     videoHemnetUrl        __typename     }\\r\\n          __typename    \\r\\n    }    threeDAttachment: attachment(type: THREE_D) {\\r\\n              id      ... on ThreeDAttachment {\\r\\n                          url        __typename      \\r\\n                          }      __typename    }\\r\\n                              isForeclosure    listingBrokerGalleryUrl    images(limit: 500) {\\r\\n                                        images {\\r\\n                                                    url(format: ITEMGALLERY_CUT)        portraitUrl: url(format: ITEMGALLERY_PORTRAIT_CUT)        fullscreenUrl: url(format: WIDTH1024)        originalHeight        originalWidth        labels        __typename      }      __typename    }    __typename  }  ... on ProjectUnit {    liveStreams(scope: ENDED) {      embedUrl      __typename    }    videoAttachment: attachment(type: VIDEO) {      id      attachmentType      ... on VideoAttachment {        videoHemnetUrl        __typename      }      __typename    }    threeDAttachment: attachment(type: THREE_D) {      id      ... on ThreeDAttachment {        url        __typename      }      __typename    }    isForeclosure    listingBrokerGalleryUrl    images(limit: 500) {      images {        url(format: ITEMGALLERY_CUT)        portraitUrl: url(format: ITEMGALLERY_PORTRAIT_CUT)        fullscreenUrl: url(format: WIDTH1024)        originalHeight        originalWidth        labels        __typename      }      __typename    }    __typename  }\\r\\n    }\\r\\n    }\",\"variables\":{\"id\":"+str(ql_id)+"}}"

headers_ql = {
  'Content-Type': 'application/json',
  'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.76 Safari/537.36'
}

response_ql = requests.request("POST", url_ql, headers=headers_ql, data=payload_ql)

#print(response_ql.text)
ql_path=response_ql.json()['data']['listing']['images']['images']

ql_df=pd.json_normalize(ql_path)
#ql_df

In [22]:
#print all images from listing item
from IPython.display import Image
for index,row in ql_df.iterrows():
   display(Image(url=row['url'], width=300))